In [1]:
%load_ext autoreload
%autoreload 2

# 1. Rendering templates from train_pbr

In [7]:
import hydra
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

# Initialize Hydra and compose the configuration
initialize(config_path="configs")
cfg = compose(config_name="run_inference")
OmegaConf.set_struct(cfg, False)


In [52]:
import os
import glob

dataset = "icbin"

default_ref_dataloader_config = cfg.data.reference_dataloader
ref_dataloader_config = default_ref_dataloader_config.copy()
ref_dataloader_config._target_ = "src.dataloader.bop_pbr.BOPTemplatePBR"
ref_dataloader_config.root_dir = f"datasets/bop23_challenge/datasets/{dataset}"
ref_dataloader_config.template_dir = " "
ref_dataloader_config.level_templates = 1 # Level 0 :42 templates, level 1, 162 templates
ref_dataloader_config.split = "train_pbr"

ply_files = glob.glob(f"datasets/bop23_challenge/datasets/{dataset}/models/models/*.ply")
obj_ids = sorted([int(os.path.basename(obj_id)[8:10]) for obj_id in ply_files])
ref_dataloader_config.obj_ids = obj_ids

out_folder = "xoa2"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
ref_dataloader_config.out_dir = out_folder
ref_dataloader_config.templates_output_folder = f"{out_folder}/real_images_templates"


In [53]:
from hydra.utils import instantiate
ref_dataset = instantiate(ref_dataloader_config) # src.dataloader.bop.BOPTemplatePBR
ref_dataset.load_processed_metaData(reset_metaData=True)

Loading metaData: 100%|██████████| 3/3 [00:00<00:00, 92.45it/s]
Finding nearest rendering close to template poses: 100%|██████████| 2/2 [00:00<00:00, 111.86it/s]


In [54]:
from torchvision.utils import save_image
import pandas as pd

templates = pd.read_csv(f"{out_folder}/train_pbr_processed_metaData.csv")

# Save templates as images
for ob_id in range(1,len(templates)//162+1): # Change to  42, 162, 462 for level templates 0, 1, 2
    obj_id_path = os.path.join(f"{out_folder}/synthetic_images_templates/icbin/train_pbr", f'obj_{(ob_id):06d}') # The transformed templates - check line 244, 245 in bop_pbr.py
    original_obj_id_path = os.path.join(f"{out_folder}/synthetic_images_templates/{dataset}/train_pbr", f'obj_{(ob_id):06d}_original')
    
    if not os.path.exists(obj_id_path):
        os.makedirs(obj_id_path)
    if not os.path.exists(original_obj_id_path):
        os.makedirs(original_obj_id_path)

    for i, img in enumerate(ref_dataset[ob_id-1]["templates"]): # all the crop her is normalize and resize to 224*224 alsready in __getitem__
        save_image(img, os.path.join(obj_id_path, f'{i:06d}.png'))
    for i, img in enumerate(ref_dataset[ob_id-1]["original_templates"]): # all the crop her is normalize and resize to 224*224 alsready in __getitem__
        save_image(img, os.path.join(original_obj_id_path, f'{i:06d}.png'))